In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
##local_path = "/your/path/here"
local_path = "/users/thudson/Documents/Dropbox/chicago-police-data/02-column_std"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/output')

['incomplete',
 'TRRs',
 'complaints-cpd-2016-dec',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-nov',
 'complaints-cpd-2016-oct',
 'complaints-ipra-2016-apr',
 'complaints-merged-2015',
 'shootings-cpd-feb2016',
 'shootings-ipra-may2016']

In [5]:
in_path = path + '/output/complaints-ipra-2016-apr/'
out_path = local_path + '/output/complaints-ipra-2016-apr/'

In [6]:
try:
    os.mkdir(out_path)
except:
    print("path exists: {}".format(out_path))

path exists: /users/thudson/Documents/Dropbox/chicago-police-data/02-column_std/output/complaints-ipra-2016-apr/


In [7]:
##doc_path = '/Data/chicago-police-data/02-column_std'
##column_dict = db.download_file(doc_path+'/doc/','Column_Dictionary.csv')

doc_path = local_path
column_dict = pd.read_csv(doc_path+'/doc/'+'Column_Dictionary.csv').iloc[:,0:2]
column_dict.head()

,Orig Column Name,New Name
0,Accused,Officer.Accused.Full_Name
1,Accused_Appointed_Date,Officer.Accused.Appointed_Date
2,Accused_Appointment_Date,Officer.Accused.Appointed_Date
3,Accused_Birth_Year,Officer.Accused.Birth_Year
4,Accused_Detail,Officer.Accused.Detail


In [8]:
## SOMETHING TO PUT IN UTILS? HOW WOULD THE SPECIFIED COLUMN_DICT WORK?

def column_mapper(df,
                  column_dict = column_dict):
    current_columns = pd.DataFrame(df.columns.tolist(),columns=['Orig Column Name'])
    new_columns = current_columns.merge(column_dict,how='left',on='Orig Column Name')['New Name'].tolist() 
    if current_columns.shape[0] == len(new_columns):
        return new_columns
    else:
        print("Join failed, more columns returned than expected")

## Apr 2016 Data

### Report 1

In [9]:
in_path_apr_2016_report = in_path
out_path_apr_2016_report = out_path

files = db.list_files(in_path_apr_2016_report)
files

['All_complaints_during_2012-2016YTD.csv',
 'All_complaints_during_2012-2016YTD.xlsx',
 'All_complaints_during_2012-2016YTD_metadata.csv',
 'IPRA_Investigators_Officers.csv',
 'IPRA_Investigators_Officers.xlsx',
 'IPRA_Investigators_Officers_metadata.csv',
 'IPRA_Investigators_Staff.csv',
 'IPRA_Investigators_Staff.xlsx',
 'IPRA_Investigators_Staff_metadata.csv',
 'Shootings_Victim_Data.csv',
 'Shootings_Victim_Data.xlsx',
 'Shootings_Victim_Data_metadata.csv']

In [10]:
file = files[0]
df = db.download_file(in_path_apr_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-ipra-2016-apr/all_complaints_during_2012-2016ytd.csv


/Users/thudson/anaconda/envs/civis-model/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['CRID',
 'Beat',
 'Address',
 'Street',
 'Address_of_Incident',
 'Location_Value',
 'Incident_Time_Start',
 'Incident_Time_End',
 'Notification_Date',
 'Closed_Date',
 'Investigator_Last_Name',
 'Investigator_First_Name',
 'Accused_First_Name',
 'Accused_Last_Name',
 'Accused_Star',
 'Accused_Unit',
 'Accused_Detail',
 'Accused_Appointment_Date',
 'Accused_Rank',
 'Initial_Category_Code',
 'Initial_Category_Description',
 'Current_Category_Code',
 'Current_Category_Description',
 'Current_Status',
 'Finding_Code',
 'Disciple_Code',
 'Discipline_Description',
 'Involved_Officer_Last_Name',
 'Involved_Officer_First_Name',
 'Involved_Officer_Unit',
 'Involved_Officer_Detail',
 'Involved_Officer_Rank',
 'Involved_Officer_Age',
 'Involved_Officer_Sex',
 'Involved_Officer_Race',
 'Report_Produced_Date']

In [16]:
df['Initial_Category_Code'].unique()

array(['20C', 'S123', '05M', '05A', 'S001', '20B', 'S013', '05K', '05R',
       '01B', 'S117', 'S008', '05B', 'S021', '05Q', '20A', '20D', '05P',
       '05Z', 'S022', 'S121', 'S119', 'S020', '04H', 'S014', 'S031',
       'S026', 'S125', '10J', 'S028', '04Z', nan, 'S002', '01C', '05C',
       'S109', '05L', 'S043', 'S249', 'S083', 'S042', 'S010', 'S006',
       '05G', 'S011', 'S029', 'S110', 'S009', '09A', 'S035', 'S196', '01Z',
       '20Z', '03D', 'S084', 'S012', 'S005', '03B', 'S007', '01A', '05N',
       'S023', '12B', 'S251', 'S253', '04D', '18A', 'S112', 'S065', 'S114',
       'S004', 'S113', 'S081', 'S044', 'S056', '02B', 'S116', 'S126',
       '19A', 'S027', '10Z', 'S101', '05D', 'S118', 'S033', '05F', 'S030',
       '18B', '05J', 'S111', '10M', '10U', 'S085', '05H', 'S094', '03G',
       '03T', '05E', '19B', '10Q', '12Z', 'S115', 'S057', 'S017', 'S068',
       'S055', '08Z', '01D', 'S025', 'S036', 'S058', 'S041', 'S032'], dtype=object)

In [20]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Incident.Location.Beat',
 'Incident.Location.Address_Nbr',
 'Incident.Location.Street',
 'Incident.Location.Address',
 'Incident.Location.Location_Value',
 'Incident.Time.Start_Time',
 'Incident.Time.End_Time',
 'Complaint.Notification_Date',
 'Complaint.Current.Closed_date',
 'Investigator.Last_Name',
 'Investigator.First_Name',
 'Officer.Accused.First_Name',
 'Officer.Accused.Last_Name',
 'Officer.Accused.Star',
 'Officer.Accused.Unit',
 'Officer.Accused.Detail',
 'Officer.Accused.Appointed_Date',
 'Officer.Accused.Rank',
 'Complaint.Initial.Category',
 'Complaint.Initial.Description',
 'Complaint.Current.Category',
 'Complaint.Current.Description',
 'Complaint.Current.Case_Status',
 'Complaint.Current.Finding',
 'Complaint.Discipline.Code',
 'Complaint.Discipline.Description',
 'Officer.Involved.Last_Name',
 'Officer.Involved.First_Name',
 'Officer.Involved.Unit',
 'Officer.Involved.Detail',
 'Officer.Involved.Rank',
 'Officer.Involved.Age',
 'Officer.Involved.Gender',
 '

In [21]:
df.to_csv(out_path_apr_2016_report+file,index=False)

### Report 2

In [26]:
file = files[3]
df = db.download_file(in_path_apr_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-ipra-2016-apr/ipra_investigators_officers.csv


['CRID',
 'Investigator_Last_Name',
 'Investigator_First_name',
 'Investigator_Rank']

In [27]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Investigator.Last_Name',
 'Investigator.First_Name',
 'Investigator.Rank']

In [28]:
df.to_csv(out_path_apr_2016_report+file,index=False)

### Report 3

In [35]:
file = files[6]
df = db.download_file(in_path_apr_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-ipra-2016-apr/ipra_investigators_staff.csv


['Investigator_Name',
 'Investigator_Rank',
 'Investigator_City_Hire_Date',
 'Investigator_IPRA_Hire_Date',
 'Investigator_Employment_Status',
 'Investigator_Left_Date']

In [36]:
df.columns = column_mapper(df)
df.columns.tolist()

['Investigator.Full_Name',
 'Investigator.Rank',
 'Investigator.City_Hire_Date',
 'Investigator.IPRA_Hire_Date',
 'Investigator.Employment_Status',
 'Investigator.Left_Date']

In [37]:
df.to_csv(out_path_apr_2016_report+file,index=False)

### Report 4

In [57]:
file = files[9]
df = db.download_file(in_path_apr_2016_report,file)
df.columns.tolist()

/Data/chicago-police-data/import/output/complaints-ipra-2016-apr/shootings_victim_data.csv


['CRID',
 'Unit',
 'Police_Shooting',
 'Incident_Date',
 'Address',
 'Direction',
 'Street',
 'Address_of_Incident',
 'City',
 'Zip',
 'Party',
 'Initial_Category_Code',
 'Initial_Category_Description',
 'Initital_Category_Type_Code',
 'Initital_Category_Type_Description',
 'Current_Category_Code',
 'Current_Category_Description',
 'Current_Category_Type_Code',
 'Current_Category_Type_Description',
 'Victim_Age',
 'Victim_Sex',
 'Victim_Race',
 'Complaint_Date',
 'CR_Required',
 'Report_Produced_Date']

In [58]:
df.columns = column_mapper(df)
df.columns.tolist()

['CRID',
 'Incident.Unit',
 'Incident.Police_Shooting',
 'Incident.Time.Date',
 'Incident.Location.Address_Nbr',
 'Incident.Location.Direction',
 'Incident.Location.Street',
 'Incident.Location.Address',
 'Incident.Location.City',
 'Incident.Location.Zip',
 'Incident.Party',
 'Complaint.Initial.Category',
 'Complaint.Initial.Description',
 'Complaint.Initial.Category_Type',
 'Complaint.Initial.Category_Type_Description',
 'Complaint.Current.Category',
 'Complaint.Current.Description',
 'Complaint.Current.Category_Type',
 'Complaint.Current.Category_Type_Description',
 'Civilian.Victim.Age',
 'Civilian.Victim.Gender',
 'Civilian.Victim.Race_Code',
 'Complaint.Date',
 'Complaint.Register_Required',
 'Report_Produced_Date']

In [59]:
df.to_csv(out_path_apr_2016_report+file,index=False)